# 门控循环单元

- 在RNN中，当时间步数较大或者时间步数较小时，循环神经网络的梯度较容易出现衰减或者爆炸。虽然梯度裁剪可以应对梯度爆炸，但是无法解决梯度衰减的问题。通常由于这个原因，循环神经网络在实际中难以捕捉时间序列中时间步距离较大的依赖关系

所以门控循环神经网络提出了：
- 重置门
- 更新门
   

## 从零开始实现

In [1]:
import sys
sys.path.append('../')

In [2]:
import gluonbook as gb
import mxnet as mx

C:\Users\lenovo\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [4]:
from mxnet import gluon,autograd,nd,init
from mxnet.gluon import loss as gloss,data as gdata,nn
from mxnet.gluon import rnn

In [5]:
(corpus_indices,char_to_idx,idx_to_char,
vocab_size) = gb.load_data_jay_lyrics()

## 初始化模型参数

In [7]:
num_inputs,num_hiddens,num_outputs = vocab_size,256,vocab_size
ctx = gb.try_gpu()

In [8]:
def get_params():
    def _one(shape):
        return nd.random.normal(scale=0.01,shape=shape,ctx=ctx)
    def _three():
        return (_one((num_inputs,num_hiddens)),
                 _one((num_hiddens,num_hiddens)),
                 nd.zeros(num_hiddens,ctx=ctx))
    
    W_xz,W_hz,b_z = _three()         #更新门参数
    W_xr,W_hr,b_r = _three()         #重置门参数
    W_xh,W_hh,b_h = _three()         #候选隐藏状态参数
    
    #输出层参数
    W_hq = _one((num_hiddens,num_outputs))
    b_q = nd.zeros(shape=(num_outputs,),ctx=ctx)
    
    #创建梯度
    params = [W_xz, W_hz, b_z, W_xr, W_hr, b_r, W_xh, W_hh, b_h, W_hq, b_q]
    for param in params:
        param.attach_grad()
    return params

## 定义模型

+ 定义隐藏状态的初始化函数，返回一个(批量大小，隐藏单元个数)的值为0的NDArray元组

In [9]:
def init_gru_state(batch_size,num_hiddens,ctx):
    return (nd.zeros(shape = (batch_size,num_hiddens),ctx=ctx),)

+ 下面根据表达式搭建 gru 的模型

In [16]:
def gru(inputs,state,params):
    #获取参数
    W_xz, W_hz, b_z, W_xr, W_hr, b_r, W_xh, W_hh, b_h, W_hq, b_q = params
    #获取状态
    H, = state
    #输入有num_stpes
    outputs = []
    for X in inputs:
        #首先计算重置门以及更新门
        #重置门
        R_t = nd.sigmoid(nd.dot(X,W_xr)+nd.dot(H,W_hr)+b_r)
        #更新门
        Z_t = nd.sigmoid(nd.dot(X,W_xz)+nd.dot(H,W_hz)+b_z)

        #计算候选隐藏层的状态
        H_t = nd.tanh(nd.dot(X,W_xh)+nd.dot(R_t*H,W_hh)+b_h)

        #更新隐藏状态
        H = Z_t*H+(1-Z_t)*H_t

        #计算输出
        Y = nd.dot(H,W_hq)+b_q
        outputs.append(Y)
    return outputs,(H,)

## 训练模型并且创作歌词

In [30]:
num_epochs, num_steps, batch_size, lr, clipping_theta = 500, 100, 32, 1e2, 1e-2
pred_period, pred_len, prefixes = 40, 50, ['我', '牵']

In [18]:
gb.train_and_predict_rnn(gru, get_params, init_gru_state, num_hiddens,
vocab_size, ctx, corpus_indices, idx_to_char,
char_to_idx, False, num_epochs, num_steps, lr,
clipping_theta, batch_size, pred_period, pred_len,
prefixes)

epoch 40, perplexity 148.773890, time 1.09 sec
 - 分开 我想你的让我想想想想想想想想想想想想想想想想想想想想想想想想想想想想想想想想想想想想想想想想想想想
 - 不分开 我想你的让我想想想想想想想想想想想想想想想想想想想想想想想想想想想想想想想想想想想想想想想想想想想
epoch 80, perplexity 32.552420, time 1.15 sec
 - 分开 我想要这样 我不要再想 我不要再想 我不要再想 我不要 爱你的美笑 像像的可爱女人 坏坏的让我疯狂
 - 不分开 爱不不觉 不知不觉 我不要再想 我不要再想 我不要 我不 我不 我不 我不 我不 我不 我不 我不
epoch 120, perplexity 5.669898, time 1.22 sec
 - 分开 一直我 说你怎么每每 我说想这样 我想要这样打我妈妈 难道你手 我有一定看看 后知后觉 又知了一个
 - 不分开 爱知不觉 你来经没不舍 连色入秋 快使用双截棍 哼哼哈兮 快使用双截棍 哼哼哈兮 快使用双截棍 哼
epoch 160, perplexity 1.726950, time 1.12 sec
 - 分开 这样的梦猫笑味河像  感不起 语沉默娘子 娘子在壶泣 我来将拖着走 静静悄悄默默离开 陷入了危险边
 - 不分开 已经 是你开的玩笑 想通 却又再考倒我 说散 你想很久了吧? 败真的黑笑幽会  有教了回对我的愿跳


## 使用GLuon接口实现

In [28]:
gru = rnn.GRU(1024)
model = gb.RNNModel(gru,vocab_size)

In [31]:
gb.train_and_predict_rnn_gluon(model, num_hiddens, vocab_size, ctx,
corpus_indices, idx_to_char, char_to_idx,
num_epochs, num_steps, lr, clipping_theta,
batch_size, pred_period, pred_len, prefixes)

epoch 40, perplexity 273.878454, time 0.30 sec
 - 我 我我我的 我我我的 我我我的 我我我的 我我我的 我我我的 我我我的 我我我的 我我我的 我我我的
 - 牵 我我我的 我我我的 我我我的 我我我的 我我我的 我我我的 我我我的 我我我的 我我我的 我我我的
epoch 80, perplexity 179.696000, time 0.30 sec
 - 我 我不 我不 我不 我不 我不 我不 我不 我不 我不 我不 我不 我不 我不 我不 我不 我不 我
 - 牵 我不 我不 我不 我不 我不 我不 我不 我不 我不 我不 我不 我不 我不 我不 我不 我不 我
epoch 120, perplexity 103.103691, time 0.30 sec
 - 我 你不再再 我不要再 我不要再 我不要再 我不要再 我不要再 我不要再 我不要再 我不要再 我不要再
 - 牵 我不要再 我不要再 我不要再 我不要再 我不要再 我不要再 我不要再 我不要再 我不要再 我不要再
epoch 160, perplexity 55.813293, time 0.30 sec
 - 我 你的完美 你在西人 一颗步步 三颗四  什么我 有你的空 快人人  什么我 有你的美 快人人  什
 - 牵 我想你的爱笑 在西人人 在使村双 一颗四四  连什么 快使我 有有  什么我 有你的空 快使用截 
epoch 200, perplexity 23.898157, time 0.30 sec
 - 我 你不再再想我不要 爱你的手不放开不能能可爱不能 爱可不可以简简单可可可女人 坏坏的让我疯狂的可爱女
 - 牵 我想要你想想你的怒火 我想揍你想想要 我不要再想你的怒火 我想揍你已经经不能 不要再再想你的怒火 
epoch 240, perplexity 9.179181, time 0.30 sec
 - 我 你说啊这样着我的妈望就怎么 就怎么么 我有多努恼恼你奔跑跑不想想想要你 这样我的太笑 一壶看酒 又
 - 牵 你想就这样着我的愿望就怎么 就怎么么 我有多努恼恼你奔跑跑不想想想要你 这样我的太笑 一壶看酒 又
epoch 280, perplexity 3.589548, time 0.30 sec
 - 我 你说啊 你怎么

## num_timesteps 越大 收敛速度越慢